In [12]:
import random
from llama_index.core.workflow import Context
from llama_index.llms.openai import OpenAI
from openai import OpenAI as OpenAI_from_openai
from dummy_data import user_information, product_information, product_reviews
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent, AgentWorkflow
from llama_index.utils.workflow import draw_all_possible_flows
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)
import json
import time
import asyncio
from datetime import datetime

# print("----------------------------------------------------------------------------")
# print("User Information: ", user_information)
# print("----------------------------------------------------------------------------")
# print("Product Information: ", product_information)
# print("----------------------------------------------------------------------------")
# print("Product Reviews: ", product_reviews)

# Initialize OpenAI LLM
llm = OpenAI(model="gpt-4o", api_key=" ")

async def retrieve_user_profile(ctx: Context) -> str:
    """Fetches user information."""
    print(f"\n$$$$$$$   [{datetime.now()}] START OF USER_PROFILE   $$$$$$$\n")

    await asyncio.sleep(3)
    current_state = await ctx.get("state", {})
    users = user_information
    user_id_to_return = 3
    current_state["user_profile"] = users[str(user_id_to_return)]
    await ctx.set("state", current_state)

    print(f"\n$$$$$$$   [{datetime.now()}] END OF USER_PROFILE   $$$$$$$\n")


    return "User profile fetched."

async def retrieve_product_information(ctx: Context) -> str:
    """Fetches product information."""
    print(f"\n$$$$$$$   [{datetime.now()}] START OF PRODUCT_INFORMATION   $$$$$$$\n")

    await asyncio.sleep(3)
    current_state = await ctx.get("state", {})
    current_state["product_information"] = product_information
    await ctx.set("state", current_state)

    print(f"\n$$$$$$$   [{datetime.now()}]  END OF PRODUCT_INFORMATION   $$$$$$$\n")

    return "Product information fetched."

async def retrieve_product_reviews(ctx: Context) -> str:
    """Fetches product reviews."""
    
    print(f"\n$$$$$$$   [{datetime.now()}]   START OF PRODUCT_REVIEWS   $$$$$$$\n")

    await asyncio.sleep(3)
    current_state = await ctx.get("state", {})
    current_state["product_reviews"] = product_reviews
    await ctx.set("state", current_state)

    print(f"\n$$$$$$$   [{datetime.now()}]   END OF PRODUCT_REVIEWS   $$$$$$$\n")


    return "Product reviews fetched."


async def features_highlighting(ctx: Context) -> str:
    """Analyzes user profile and product data to highlight relevant features of the product based on the user profile."""
    print(f"\n$$$$$$$   [{datetime.now()}]   START OF FEATURES_HIGHLIGHTING   $$$$$$$\n")

    await asyncio.sleep(3)
    current_state = await ctx.get("state", {})
    user_profile = current_state.get("user_profile", {})
    product_info = current_state.get("product_information", {})

    prompt_template = f"""
    You are given a user profile and product information. Analyze the details of both and suggest which features of the product will be most appealing to the user. As your final output, return only a list of features. Do not put a variable name before the list. Your response should only contain the list and nothing else:
    
    [
        **List of featrures selected**
    ]
    
    Perform the task for the following information:
    User Profile: {user_profile}
    Product Information: {product_info}
    """
    
    client = OpenAI_from_openai(api_key=" ")
    messages = [{"role": "user", "content": prompt_template}]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0.1,
        top_p=0.1,
        messages=messages
    ).choices[0].message.content.strip()

    # response_in_json_object = json.loads(response)

    current_state["suggested_features"] = response
    await ctx.set("state", current_state)

    print(f"\n$$$$$$$   [{datetime.now()}]   END OF FEATURES_HIGHLIGHTING   $$$$$$$\n")


    return "Features suggested."

async def sort_reviews(ctx: Context) -> str:
    """Analyzes user profile and the list of reviews and sort the reviews according to the user profile. The reviews that are most likely to be of interest to the user should be coming first in the list of reviews."""
    print(f"\n$$$$$$$   [{datetime.now()}]   START OF SORT REVIEWS   $$$$$$$\n")

    await asyncio.sleep(3)
    current_state = await ctx.get("state", {})
    user_profile = current_state.get("user_profile", {})
    product_reviews = current_state.get("product_reviews", [])

    prompt_template = f"""
    You are given a user profile and a list of product reviews. Analyze the details of both and sort the reviews based on the user profile information. The reviews that match the users profile the most and are the ones that the user will most likely be interested in should come first. As your final output, return only a list of reviews sorted based on the user profile information. Do not put a variable name before the list. Your response should only contain the list and nothing else:
    
    [
        **List of reviews sorted**
    ]

    User Profile: {user_profile}
    Product Reviews: {product_reviews}
    """
    
    client = OpenAI_from_openai(api_key=" ")
    messages = [{"role": "user", "content": prompt_template}]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0.1,
        top_p=0.1,
        messages=messages
    ).choices[0].message.content.strip()

    current_state["sorted_reviews"] = response
    await ctx.set("state", current_state)

    print(f"\n$$$$$$$   [{datetime.now()}]   END OF SORT_REVIEWS   $$$$$$$\n")


    return "reviews sorted."

async def custom_description(ctx: Context) -> str:
    """Analyzes user profile and product data to create a custom product description that will be most intriguing to the user."""
    print(f"\n$$$$$$$   [{datetime.now()}]  START OF CUSTOM_DESCRIPTION   $$$$$$$\n")
    
    await asyncio.sleep(3)
    current_state = await ctx.get("state", {})
    user_profile = current_state.get("user_profile", {})
    product_info = current_state.get("product_information", {})

    prompt_template = f"""
    You are given a user profile and product information. You have to create a custom description of the product based on the user profile information. The description should be most appealing to the user and should not be longer than 1 sentence.  As your final output, return a json object ONLY, like the follwowing. Do not put a variable name before the json object. Your response should only contain the json object and nothing else:
    {{
        "custom_description": **The custom description that you create**
    }}
    
    User Profile: {user_profile}
    Product Information: {product_info}
    """
    
    client = OpenAI_from_openai(api_key=" ")
    messages = [{"role": "user", "content": prompt_template}]
    
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0.1,
        top_p=0.1,
        messages=messages
    ).choices[0].message.content.strip()

    current_state["custom_description"] = response
    await ctx.set("state", current_state)

    print(f"\n$$$$$$$   [{datetime.now()}]   END OF CUSTOM_DESCRIPTION   $$$$$$$\n")
    return "created custom description."

async def hand_off_to_agent(ctx: Context) -> str:
    """Hands off the control to another agent."""

    return "You should hand off the task to appropriate agents and run them in Parallel."


# Define Agents
from llama_index.core.agent.workflow import FunctionAgent
user_data_agent = FunctionAgent(
    name="UserDataFetchingAgent",
    description="Used to retrieve user information",
    system_prompt="""You are an agent who has tools to fetch data. You have the following tools: 
    - retrieve_user_profile: this tool is used to fetch user information. Use this when you have to fetch the user details.
    
    All of these tools perform tasks independently. Call whatever tool you need to execute the current at hand task. You can call them in parallel because they execute independent of each other.
    """,
    llm=llm,
    tools=[retrieve_user_profile],
    can_handoff_to=["ProductDataFetchingAgent"],
    # allow_parallel_tool_calls=True,
)

product_data_agent = FunctionAgent(
    name="ProductDataFetchingAgent",
    description="Used to fetch product data.",
    system_prompt="""You are an agent that is used to make retrieve product data. You have the following tools.
    - retrieve_product_information: this tool is used to fetch product information. Use this when you have to fetch the product details.
    - retrieve_product_reviews: this tool is used to fetch product reviews. Use this when you have to fetch the product reviews.
    
    These tools are used for customization based on the user profile. Use the 'features_highlighting' tool to suggest which features should be highlighted. Use the 'sort_reviews' tool to sort the reviews based on the user profile. Use the 'custom_description' tool to create a custom product description based on the user profile.

    All of these tools perform tasks independently. You can call them in parallel. Call whatever tool you need to perform the current task at hand.

    """,
    llm=llm,
    tools=[retrieve_product_information, retrieve_product_reviews],
    can_handoff_to=["CustomizationsAgent"],
)
customizations_agent = FunctionAgent(
    name="CustomizationsAgent",
    description="Used to customize product data",
    system_prompt="""You are an agent that is used to make customizations like suggest product features to be highlighted and create custom product descriptions. You have the acces to followng agents.
    - DescriptionCustomizationsAgent: this agent is used to customize product description. Use this when you have to create customized product description.
    - FeaturesCustomizationsAgent: this agent is used to suggest product features. Use this when you have to suggest product features.
    
     The agents are used for customization based on the user profile. Use the 'FeaturesCustomizationsAgent' agent to suggest which features should be highlighted. Use the 'DescriptionCustomizationsAgent' agent to create a custom product description based on the user profile.

    All of these agents perform tasks independently. You can call them in parallel. Call whatever agent you need to perform the current task at hand.

    """,
    llm=llm,
    tools=[hand_off_to_agent],
    can_handoff_to=["FeaturesCustomizationsAgent","DescriptionCustomizationsAgent"],
)

features_customizations_agent = FunctionAgent(
    name="FeaturesCustomizationsAgent",
    description="Used to customize product data",
    system_prompt="""You are an agent that is used to make customizations based on the user profile. You have the following tools.
    - features_highlighting: this tool is used to suggest which features should be highlighted. Based on the users profile, this tool will suggest which features of the product should be highlighted. Call this tool for features_highlighting.

    """,
    llm=llm,
    tools=[features_highlighting],
    can_handoff_to=["CustomizationsAgent"],
)

description_customizations_agent = FunctionAgent(
    name="DescriptionCustomizationsAgent",
    description="Used to customize product data",
    system_prompt="""You are an agent that is used to make custom product descriptions based on the user profile. You have the following tools.
    - custom_description: this tool is used to create a custom product description based on the user profile. Call this tool to create a custom product description.
   
    """,
    llm=llm,
    tools=[custom_description],
    can_handoff_to=["CustomizationsAgent"],

)

# Create Agent Workflow
agent_workflow = AgentWorkflow(
    agents=[user_data_agent, product_data_agent, customizations_agent],
    root_agent=user_data_agent.name,
    initial_state={
        "user_profile": {},
        "product_information": {},
        "product_reviews": [],
    },
)

# from llama_index.utils.workflow import draw_all_possible_flows

# draw_all_possible_flows(agent_workflow, filename="1.html")

# Run the workflow (if running as a script)
import nest_asyncio
nest_asyncio.apply()

start = time.time()
# asyncio.run(run_workflow())


handler = agent_workflow.run(
    user_msg="""

    Fetch the users details and fetch the product information the suggest product features to be highlighted and create a custom product description in parallel.
    
    
    """
)


current_agent = None
async for event in handler.stream_events():
    if hasattr(event, "current_agent_name") and event.current_agent_name != current_agent:
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")
    
    if isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print("🛠️  Planning to use tools:", [call.tool_name for call in event.tool_calls])
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 [{datetime.now()}] Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")

state = await handler.ctx.get("state")

end = time.time()
print("\n\n**** Time taken: ", end - start)

print("Keys:  ", state.keys())

# file_path = "formatted_data.json"

# with open(file_path, "w", encoding="utf-8") as file:
#     json.dump(state, file, indent=4, ensure_ascii=False)




🤖 Agent: UserDataFetchingAgent

🛠️  Planning to use tools: ['retrieve_user_profile', 'handoff']

$$$$$$$   [2025-02-20 20:49:51.558304] START OF USER_PROFILE   $$$$$$$

🔨 [2025-02-20 20:49:51.559302] Calling Tool: retrieve_user_profile
  With arguments: {}
🔨 [2025-02-20 20:49:51.560303] Calling Tool: handoff
  With arguments: {'to_agent': 'ProductDataFetchingAgent', 'reason': 'Used to fetch product data.'}
🔧 Tool Result (handoff):
  Arguments: {'to_agent': 'ProductDataFetchingAgent', 'reason': 'Used to fetch product data.'}
  Output: Agent ProductDataFetchingAgent is now handling the request due to the following reason: Used to fetch product data..
Please continue with the current request.

$$$$$$$   [2025-02-20 20:49:54.572614] END OF USER_PROFILE   $$$$$$$

🔧 Tool Result (retrieve_user_profile):
  Arguments: {}
  Output: User profile fetched.

🤖 Agent: ProductDataFetchingAgent

🛠️  Planning to use tools: ['retrieve_product_information', 'handoff']

$$$$$$$   [2025-02-20 20:49:56.297